In [69]:
#import numpy as np
import math
from collections import Counter

با سلام و خسته نباشید. در ادامه توضیحات کد را مشاهده میکنید.
کتابخوانه های مورد نیاز
(نامپای را همیشه برحسب عادت مینویسم و کپشن میکنم وگرنه در این تمرین اصلا از نامپای استفاده نشده است.)

In [72]:
#define query n documents
q = "machine learning"
d1 = "Machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning. Machine learning is a subset of artificial intelligence that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. The primary aim is to allow computers to learn automatically without human intervention or assistance and adjust actions accordingly. Machine learning focuses on the development of computer programs that can access data and use it to learn for themselves."
d2 = "Technology has revolutionized every aspect of modern life. From smartphones to advances in biotechnology, the impact is profound. Machine learning plays a crucial role in data analysis, predictive modeling, and automation. Other technologies like blockchain, cloud computing, and quantum computing are also emerging rapidly. The integration of these technologies is shaping the future of industries worldwide. In manufacturing, machine learning algorithms optimize production lines. In healthcare, machine learning aids in diagnosing diseases. The transportation sector uses machine learning for autonomous vehicles. With the growth of big data, the importance of machine learning continues to increase. Ethical considerations in machine learning are becoming more prominent as the technology advances."

print("Query is:", q)
print("Document1 is:", d1)  
print("Document2 is:", d2)
docs = [d1, d2]

Query is: machine learning
Document1 is: Machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning. Machine learning is a subset of artificial intelligence that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. The primary aim is to allow computers to learn automatically without human intervention or assistance and adjust actions accordingly. Machine learning focuses on the development of computer programs that can access data and use it to learn for themselves.
Document2 is: Technology has revolutionized every aspect of modern life. From smartphones to advances in biotechnology, the impact is profound. Machine learning plays a crucial role in data analysis, predictive modeling, and automation. Other technologies like blockchain, cloud computing, and quantum computing are also emerging rapidly. The integration

محتوای کوئری درون رشته کیو ریخته شده است
همچنین محتوای داکیومنت های 1 و 2 نیز درون رشته مربوطه قرار گرفتند 
در انتها هریک را در خروجی چاپ کردیم.
همچنین دو سند متنی را در یک لیست قرار دادم.

In [75]:
#calcu PLN func
def norm_len(doc, k=0.5):
    terms = doc.lower().split()
    tf = Counter(terms)
    N = len(terms)
    norm_length = 0
    
    for term, freq in tf.items():
        norm_length += (freq / N) * (1 - k) + k
    return norm_length

تابع نورم-لن طول نرمال شده اسناد را محاسبه میکند
تمامی ترم ها و کلمات هر سند نیز بصورت یک لیست از کلمات با حروف کوچیک در میان
در اخر هم برای هر کلمه و فرکانسش، طول نرمال شده با استفاده از فرمولی که نوشته شده محاسبه میشه.

In [78]:
#calcu PLN for each doc
def rank_docs_pivoted(q, docs):
    q_terms = q.lower().split()
    scores = []
    
    for doc in docs:
        doc_terms = doc.lower().split()
        doc_score = 0
        for term in q_terms:
            doc_score += norm_len(doc)
        scores.append(doc_score)
    
    return scores


کوئری که تبدیل به ترم های مختلف شد. بعدش برای هر سند طول نرمال  رو با تابع نورم-لن حساب میکنیم

In [81]:
#Calcu Rank base on PLN scores of each doc
pivoted_scores = rank_docs_pivoted(q, docs)
print("Pivoted Normalization Scores:", pivoted_scores)

#calcu BM25 func
def bm25_score(q, doc, k1=1.5, b=0.75):
    terms = doc.lower().split()
    tf = Counter(terms)
    doc_len = len(terms)
    avg_dl = sum(len(d.lower().split()) for d in docs) / len(docs)
    score = 0
    
    for term in q.lower().split():
        if term in tf:
            tf_val = tf[term]
            idf = math.log((len(docs) - sum(1 for d in docs if term in d.lower().split()) + 0.5) / (sum(1 for d in docs if term in d.lower().split()) + 0.5))
            score += idf * ((tf_val * (k1 + 1)) / (tf_val + k1 * (1 - b + b * (doc_len / avg_dl))))
    
    return score


Pivoted Normalization Scores: [48.99999999999996, 77.99999999999993]


مقدار پیوت نرمال شده برای هر سند رو پرینت میکنیم توی خروجی
و در ادامه قسمت بعدی از سوال که مربوط به بی ام 25 میشه رو انجام میدیم

تابع بی ام رو بر اساس کوئری محاسبه میکنیم که 
طول سند، طول متوسط سند و تابع ای دی اف برای هر کلمه داخلش قرار میدیم که حساب کنه
در نهایت مقدار عددی بی ام 25 سند با فرمول محاسبه میشه.

In [88]:
#calcu doc score base on BM25
def rank_docs_bm25(q, docs):
    scores = []
    for doc in docs:
        score = bm25_score(q, doc)
        scores.append(score)
    
    return scores

#prepare score for docs base od BM25
bm25_scores = rank_docs_bm25(q, docs)
print("BM25 Scores are:", bm25_scores)

BM25 Scores are: [-7.198941084849965, -6.3137898079571375]


مقدار عددی بی ام 25 رو برای تمام اسناد محاسبه میکنیم که کلا 2 تا سند داریم
در انتها هم مقدار عددی اونارو در خروجی پرینت میکنیم.